In [1]:
import os
from openai import OpenAI


# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.siliconflow.cn/v1",
   
)

In [2]:
import re


def split_legal_documents(text):
    """
    优化版法律条文分割函数：
    1. 精确分割独立条文（含多款条文）
    2. 保留层级结构信息
    3. 自动识别条文中的多款内容
    """
    # 匹配格式：**第X条** + 内容（支持多款）
    pattern = r"\*\*(第[\u4e00-\u9fa5]{2,}条)\*\*\s*((?:[^※]+?(?=\*\*第|$))?)"
    articles = []
    
    matches = re.finditer(pattern, text, re.DOTALL)
    for match in matches:
        article_num = match.group(1)  # 条文编号（如"第二百三十四条"）
        content = match.group(2).strip()
        
        # 检测多款结构（按自然段分割）
        clauses = [c.strip() for c in re.split(r'\n{2,}', content) if c.strip()]
        
        # 单条文多款处理
        if len(clauses) > 1:
            for idx, clause in enumerate(clauses, 1):
                articles.append({
                    "article_num": f"{article_num}第{idx}款",
                    "content": clause,
                    "full_text": f"**{article_num}第{idx}款** {clause}"
                })
        else:
            articles.append({
                "article_num": article_num,
                "content": content,
                "full_text": f"**{article_num}** {content}"
            })
    
    return articles

In [3]:


with open("./mfd.md", "r") as file:
    file_text = file.read()
articles = split_legal_documents(file_text)
print(articles)

[{'article_num': '第二百零四条', 'content': '为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。', 'full_text': '**第二百零四条** 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。'}, {'article_num': '第二百零五条', 'content': '本编调整因物的归属和利用产生的民事关系。', 'full_text': '**第二百零五条** 本编调整因物的归属和利用产生的民事关系。'}, {'article_num': '第二百零六条', 'content': '国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。\n国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。\n国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。', 'full_text': '**第二百零六条** 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。\n国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。\n国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。'}, {'article_num': '第二百零七条', 'content': '国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。', 'full_text': '**第二百零七条** 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。'}, {'article_num': '第二百零八条', 'content': '不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。', 'full_text': '**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。'}, {'article_num': '第二百零九条', 'content': '不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。\n依法属于国家所有的自然资源，所有权可以不登记。', 'full_tex

In [4]:
print("Total lines:", len(articles))

Total lines: 414


In [6]:
from sentence_transformers import SentenceTransformer

# 设置代理
os.environ["HTTP_PROXY"] = "socks5h://localhost:1080"  # HTTP 代理
os.environ["HTTPS_PROXY"] = "socks5h://localhost:1080"  # HTTPS 代理

# 验证代理是否生效（可选）
try:
    import requests
    print("当前IP:", requests.get("https://ipinfo.io/json", timeout=5).json()['ip'])
except Exception as e:
    print("代理验证失败:", e)

embedding_model = SentenceTransformer("BAAI/bge-large-zh-v1.5")

当前IP: 66.98.125.172


In [8]:
print(embedding_model.smart_apply)

<bound method PreTrainedModel.initialize_weights.<locals>.smart_apply of SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)>


In [7]:
test_embedding = embedding_model.encode("Hello, world!")
print(test_embedding)

[-0.01079991  0.00639567 -0.07986796 ... -0.03491411  0.00882329
 -0.0073152 ]


In [8]:
print(f"向量维度: {test_embedding.shape}")

向量维度: (1024,)


In [9]:
embedding_list = test_embedding.tolist()
print(embedding_list)

[-0.010799910873174667, 0.006395666394382715, -0.07986795902252197, 0.005244245287030935, 0.026246629655361176, 0.06408879160881042, -0.03378396853804588, 0.01285045687109232, 0.019259637221693993, -0.03794187679886818, -0.004530563950538635, 0.03222888335585594, 0.03721104562282562, 0.012160850688815117, -0.05119388923048973, 0.05168244242668152, 0.06708357483148575, 0.00677478825673461, 0.010185017250478268, -0.03221588954329491, 0.03344046697020531, 0.01432041171938181, -0.023389311507344246, 0.025816136971116066, 0.033993884921073914, -0.00881813745945692, -0.011395705863833427, 0.005604533012956381, 0.026370931416749954, -0.01492361444979906, -0.021367272362113, -0.0032590117771178484, -0.031682319939136505, -0.0044360351748764515, -0.010650846175849438, 0.018701422959566116, -0.002389027737081051, 0.029954951256513596, 0.02695298194885254, 0.037006352096796036, -0.004308691248297691, -0.022685924544930458, 0.003978044260293245, -0.04618897661566734, 0.02868851274251938, 0.0336357

In [ ]:
embedding_dim = len(test_embedding)
print(embedding_dim)

In [10]:
from pymilvus import MilvusClient, DataType

def setup_milvus_collection(collection_name,dimension,metric_type):
    """创建并配置Milvus集合"""
    milvus_client = MilvusClient(uri="./milvus_mfd.db")

    # 删除现有集合（如果存在）
    if milvus_client.has_collection(collection_name):
        milvus_client.drop_collection(collection_name)

    # 创建带优化的集合
    milvus_client.create_collection(
        collection_name=collection_name,
        dimension=dimension,  # 根据模型维度设置（bge=1024）
        metric_type=metric_type,  # 余弦相似度
        auto_id=True,  # 自动生成ID
        description="民法典条文向量库",
        # 高级配置
        field_params=[
            {"name": "id", "type": DataType.INT64, "is_primary": True, "auto_id": True},
            {"name": "vector", "type": DataType.FLOAT_VECTOR, "dim": 1024},
            {"name": "article_num", "type": DataType.VARCHAR, "max_length": 20},
            {"name": "content", "type": DataType.VARCHAR, "max_length": 5000},
            {"name": "full_text", "type": DataType.VARCHAR, "max_length": 5000}
        ],
        index_params={
            "index_type": "IVF_FLAT",  # 精确索引
            "metric_type": "COSINE",
            "params": {"nlist": 2048}  # 聚类中心数
        }
    )
    return milvus_client


In [11]:
milvus_client= setup_milvus_collection("PRCCivilCode", 1024, "COSINE")

In [12]:
print(milvus_client)

In [15]:
batch_size = 100
for i in range(0, len(articles), batch_size):
    batch = articles[i:i+batch_size]
        
    # 生成嵌入向量
    contents = [art["content"] for art in batch]
    embeddings = embedding_model.encode(contents).tolist()
    # 准备插入数据
    data = [{
            "vector": emb,
            "article_num": art["article_num"],
            "content": art["content"],
            "full_text": art["full_text"]
        } for emb, art in zip(embeddings, batch)]
        
        # 插入Milvus
    res = milvus_client.insert("PRCCivilCode", data)
    print(f"插入批次 {i//batch_size}: {len(res['ids'])} 条记录")

    # # 创建索引（加速搜索）
    # milvus_client.create_index(
    #     "PRCCivilCode",
    #     field_name="vector",
    #     index_params={
    #         "index_type": "IVF_SQ8",
    #         "metric_type": "COSINE",
    #         "params": {"nlist": 2048}
    #     }
    # )



插入批次 0: 100 条记录
插入批次 1: 100 条记录
插入批次 2: 100 条记录
插入批次 3: 100 条记录
插入批次 4: 14 条记录


In [31]:
question = "拾得漂流物、失散的饲养动物,如何处理"
search_res = milvus_client.search(
    collection_name="PRCCivilCode",
    data=embedding_model.encode(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=5,  # 返回前5个结果
    search_params={"metric_type": "COSINE", "params": {}},  # 内积距离
    output_fields=["content","article_num","full_text"],  # 返回 text 字段
)

In [32]:
search_res[0]

[{'id': 458535311012790726, 'distance': 0.7288655042648315, 'entity': {'article_num': '第三百五十二条第1款', 'content': '拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。', 'full_text': '**第三百五十二条第1款** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。'}}, {'id': 458535310733607146, 'distance': 0.7288655042648315, 'entity': {'article_num': '第二百三十八条', 'content': '拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。', 'full_text': '**第二百三十八条** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。'}}, {'id': 458535301032706082, 'distance': 0.7288655042648315, 'entity': {'article_num': '第二百三十八条', 'content': '拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。', 'full_text': '**第二百三十八条** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。'}}, {'id': 458535311012790722, 'distance': 0.5815781950950623, 'entity': {'article_num': '第三百四十八条', 'content': '拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。', 'full_text': '**第三百四十八条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。'}}, {'id': 458535310733607142, 'distance': 0.5815781950950623, 'entity': {'article_num': '第二百三十四条', 'content': '拾得遗失物，应

In [34]:
search_res[0][4]["entity"]["content"]

'拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。'

In [63]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["content"],res["entity"]["article_num"],res["entity"]["full_text"],res["distance"]) for res in search_res[0]
]
print(retrieved_lines_with_distances)

[('拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。', '第三百五十二条第1款', '**第三百五十二条第1款** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。', 0.7288655042648315), ('拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。', '第二百三十八条', '**第二百三十八条** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。', 0.7288655042648315), ('拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。', '第二百三十八条', '**第二百三十八条** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。', 0.7288655042648315), ('拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。', '第三百四十八条', '**第三百四十八条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。', 0.5815781950950623), ('拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。', '第二百三十四条', '**第二百三十四条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。', 0.5815781950950623)]


In [69]:
context = "\n".join(
    [line_with_distance[2] for line_with_distance in retrieved_lines_with_distances]
)

In [70]:
context

'**第三百五十二条第1款** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。\n**第二百三十八条** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。\n**第二百三十八条** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。\n**第三百四十八条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。\n**第二百三十四条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。'

In [74]:
SYSTEM_PROMPT = """
Human: 你是一个经验丰富的法律工作者。你能够从提供的上下文段落片段中找到问题的答案，并给出简洁明了的总结。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题,并说清楚法律来源。；最后追加原始回答的英文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [75]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题,并说清楚法律来源。；最后追加原始回答的英文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n**第三百五十二条第1款** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。\n**第二百三十八条** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。\n**第二百三十八条** 拾得漂流物、失散的饲养动物或者受到损害的财产，参照适用拾得遗失物的有关规定。\n**第三百四十八条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。\n**第二百三十四条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。\n</context>\n<question>\n拾得漂流物、失散的饲养动物,如何处理\n</question>\n<translated>\n</translated>\n'

In [76]:
response = deepseek_client.chat.completions.create(
    # model="deepseek-chat",
    model="Pro/deepseek-ai/DeepSeek-V3",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的法律条文，处理拾得漂流物或失散的饲养动物的方式如下：

1. **法律依据**：
- 主要依据《民法典》第352条第1款（或第238条）：明确规定拾得漂流物、失散的饲养动物应参照适用拾得遗失物的规定。
- 对应适用条款为第348条（或第234条）：要求拾得遗失物应返还权利人，并及时通知领取或送交公安等部门。

2. **处理方式**：
   - **返还原主义务**：拾得人必须将物品/动物归还给权利人（所有者）
   - **通知或移交**：
     a) 直接通知权利人领取，或
     b) 送交公安机关等有权部门处理

3. **特殊说明**：
   该规定将漂流物、失散动物与遗失物作同等法律对待，适用相同处理程序。

<translated>
According to the provided legal provisions, the handling of found flotsam or stray domestic animals is as follows:

1. **Legal Basis**:
- Mainly based on Article 352(1) (or Article 238) of the Civil Code: Explicitly states that found flotsam or stray domestic animals shall be handled by reference to regulations concerning lost property.
- Corresponding applicable clause is Article 348 (or Article 234): Requires the finder to return lost property to the obligee and promptly notify them to claim it or deliver it to public security authorities.

2. **Handling Methods**:
   - **Return Obligation**: Finder must return the item/animal to the obligee (owner)
   -